In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from prophet.plot import plot_plotly
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric

# Load the data
# Assuming the data is in a CSV file named 'delivery_data.csv'
df = pd.read_csv('Zomato_data.csv')

# Convert Order_Date to datetime
df['Order_Date'] = pd.to_datetime(df['Order_Date'])

# Prepare the data for Prophet
prophet_df = df.groupby('Order_Date').size().reset_index(name='order_count')
prophet_df = prophet_df.rename(columns={'Order_Date': 'ds', 'order_count': 'y'})

# Create and fit the Prophet model
model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True)
model.fit(prophet_df)

# Create a dataframe for future predictions (e.g., next 30 days)
future_dates = model.make_future_dataframe(periods=30, freq='D')

# Make predictions
forecast = model.predict(future_dates)

# Plot the forecast
fig1 = model.plot(forecast)
plt.title('Demand Forecast')
plt.xlabel('Date')
plt.ylabel('Number of Orders')
plt.show()

# Plot the components of the forecast
fig2 = model.plot_components(forecast)
plt.show()

# Print the forecast for the next 30 days
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(30))

# Perform cross-validation
df_cv = cross_validation(model, initial='730 days', period='30 days', horizon='30 days')

# Calculate performance metrics
df_p = performance_metrics(df_cv)
print(df_p)

# Plot the cross-validation results
fig3 = plot_cross_validation_metric(df_cv, metric='mape')
plt.show()

# Additional analysis: Impact of external factors on demand

# Weather conditions
weather_impact = df.groupby('Weather_conditions')['Time_taken (min)'].mean().sort_values(ascending=False)
plt.figure(figsize=(10, 6))
weather_impact.plot(kind='bar')
plt.title('Average Delivery Time by Weather Condition')
plt.xlabel('Weather Condition')
plt.ylabel('Average Time Taken (min)')
plt.xticks(rotation=45)
plt.show()

# Road traffic density
traffic_impact = df.groupby('Road_traffic_density')['Time_taken (min)'].mean().sort_values(ascending=False)
plt.figure(figsize=(10, 6))
traffic_impact.plot(kind='bar')
plt.title('Average Delivery Time by Road Traffic Density')
plt.xlabel('Road Traffic Density')
plt.ylabel('Average Time Taken (min)')
plt.xticks(rotation=45)
plt.show()

# Festival impact
festival_impact = df.groupby('Festival')['Time_taken (min)'].mean().sort_values(ascending=False)
plt.figure(figsize=(10, 6))
festival_impact.plot(kind='bar')
plt.title('Average Delivery Time by Festival')
plt.xlabel('Festival')
plt.ylabel('Average Time Taken (min)')
plt.xticks(rotation=45)
plt.show()

# Interactive plot using plotly
fig = plot_plotly(model, forecast)
fig.show()

# Save the model
model.save('demand_forecast_model.json')

/Users/dylanmoraes/Documents/GitHub/MumbaiHacks/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


FileNotFoundError: [Errno 2] No such file or directory: 'delivery_data.csv'